In [5]:
# Celda 1: Imports, parámetros, configuración de la semilla y creación de la carpeta de archivos ───────────────────────────────────────

import numpy as np                      # Celdas: 2,3,4,5,6,7,8 (NP arrays, random choice with rng)
import matplotlib.pyplot as plt         # Celdas: 2,6,7,8 (Visualización, gráficos, animación)
import time                             # Celda: 5 (Medición de tiempos)
from tqdm import tqdm                   # Celda: 5 (Barra de progreso, opcional)
import argparse                         # Celdas: 1 (parseo de argumentos)
import sys                              # Para obtener argv en parse_known_args
from numba import njit, set_num_threads, get_num_threads, prange
import os                               # Para obtener el número de hilos disponibles
import h5py  # Para guardar resultados en formato HDF5
import subprocess, threading, re, math # Última celda para la generación del video

# ─── Parámetros del modelo ────────────────────────────────────────────────────
L        = int(500)                     #(int)
J        = 1.0                          #(float) 
T        = 10.0                         #(float)
n_sweeps = int(500)                     #(int)
threads_percentage = int(100)           #(int) Porcentaje de hilos a usar (100% = todos los disponibles)

# ─── Configuración de semilla para reproducibilidad ──────────────────────────
seed = None                             # None = usar entropía del sistema
rng  = np.random.default_rng(seed)      # PCG64 RNG: seguro y adecuado para simulaciones

#Establecemos el número de hilos a usar asegurándonos de que no exceda el número de hilos disponibles ni sea menor a 1
n_threads_available = os.cpu_count()
if n_threads_available is None:
    n_threads_available = 1  # Si no se puede determinar, usar al menos 1 hilo
threads_percentage = max(1, min(100, threads_percentage))
set_num_threads(int(n_threads_available*(threads_percentage / 100.0)))
n_threads = get_num_threads()
print(f"Usando {n_threads} hilos de {n_threads_available} disponibles ({threads_percentage}%).")

Carpeta = "results"  # Nombre de la carpeta para guardar los resultados
if not os.path.exists(Carpeta):
    os.makedirs(Carpeta)  # Crear la carpeta si no existe


Usando 16 hilos de 16 disponibles (100%).


In [ ]:
# ─── Celda 2: Inicialización de la red y visualización ───────────────────────

# Generar configuración inicial aleatoria de espines ±1
def random_config(L, rng):
    """
    Crea una matriz LxL de espines aleatorios ±1 usando el RNG proporcionado.
    """
    return rng.choice([1, -1], size=(L, L))

# Forzar que la fila de arriba sea negativa y la de abajo positiva
def fix_boundary_conditions(config):
    """
    Fuerza las condiciones de frontera: fila superior -1, fila inferior +1.
    """
    config[0, :] = -1  # Fila superior
    config[L-1, :] = 1  # Fila inferior
    return config

# Ahora creamos una función para guardar la configuración inicial en un archivo .png, y que devuelva la configuración
def init_config(destino, L, rng):
    """
    Guarda la configuración inicial en un archivo .png.
    """
    config = fix_boundary_conditions(random_config(L, rng))  # Generar configuración aleatoria y fijar condiciones de frontera
    plt.figure(figsize=(5, 5))
    plt.imshow(config, cmap='gray', interpolation='nearest')
    plt.title('Configuración inicial aleatoria')
    plt.axis('off')
    plt.savefig(destino, dpi=300, bbox_inches='tight')
    plt.close()
    return config


In [8]:
# ─── Celda 3: Definición de observables termodinámicos ─────────────────────

def energy(config, J=J):
    """
    Calcula la energía total del modelo de Ising 2D con contorno periódico.
    """
    # Enlaces derecha e inferior para contar cada par una sola vez
    right = np.roll(config, -1, axis=1)
    down  = np.roll(config, -1, axis=0)
    return -J * np.sum(config * (right + down))


def magnetization(config):
    """
    Calcula la magnetización total del sistema.
    """
    return np.sum(config)


In [ ]:
# ─── Celda 4: Funciones del propio algoritmo de ising-kawasaki ──────────────

def calculate_acceptance(frames: np.ndarray) -> np.ndarray:
    
    nframes, H, W = frames.shape
    # `True` donde el espín cambió respecto al sweep anterior
    changes = frames[1:] != frames[:-1]               # shape (nframes-1, H, W)
    diff_counts = changes.reshape(nframes-1, -1).sum(axis=1)
    # Cada swap válido intercambia dos posiciones
    accepted_swaps = diff_counts / 2
    # Nº de intentos de swap por sweep ≈ H*W
    attempts = H * W
    acceptance = accepted_swaps / attempts
    return acceptance


@njit
def delta_E_kawasaki(config, i, j, k, l, J=J):
    """
    Calcula el cambio de energía ΔE para un intercambio de espines en la dinámica de Kawasaki.
    """
    #Calculamos la energía de la configuración inicial
    E_1 = -J*(config[i,j]*(config[i,(j-1)%L] + config[(i-1)%L,j] + config[(i+1)%L,j] + config[i,(j+1)%L])+config[k,l]*(config[k,(l-1)%L] + config[(k-1)%L,l] + config[(k+1)%L,l] + config[k,(l+1)%L]))
   
    #Calculamos la energía de la configuración final
    E_2 = -J*(config[k,l]*(config[i,(j-1)%L] + config[(i-1)%L,j] + config[(i+1)%L,j] + config[i,(j+1)%L])+config[i,j]*(config[k,(l-1)%L] + config[(k-1)%L,l] + config[(k+1)%L,l] + config[k,(l+1)%L]))
    #Calculamos el cambio de energía
    delta_E = E_2 - E_1
    return delta_E


#Paso de la simulación

@njit   
def sweep_kawasaki(config, L, J, T):
    for k in prange(L*L):
        #Seleccionamos un espín aleatorio (i, j) de la red excluyendo las filas superior e inferior
        i, j = np.random.randint(1, L-1), np.random.randint(0, L)
        # Definimos los offsets para los vecinos (arriba, abajo, izquierda, derecha)
        offsets = np.array([(1, 0), (0, 1), (0, -1),  (-1, 0)], dtype=np.int64)
        # Ahora seleccionamos un offset aleatorio que decidirá si escogemos un vecino arriba, abajo, izquierda o derecha
        #Hay que mantener la condición de los espines superior e inferior.
        # Entonces lo que hacemos es limitar los offsets a 3 si estamos en la fila superior o inferior, y a 4 si estamos en el resto de la red.
        # Y luego forzamos que si está en la fila
        if i == 1:
            di, dj = offsets[np.random.randint(0, 3)]
        elif i == L-2:
            di, dj = offsets[np.random.randint(1, 4)]
        else:
            di, dj = offsets[np.random.randint(0, 4)]
        # Ahora podemos calcular la posición exacta del espín vecino
        ni, nj = (i + di) % L, (j + dj) % L
        # Ahora que tenemos la posición del espín vecino, podemos calcular el ΔE
        delta_E = delta_E_kawasaki(config, i, j, ni, nj, J)
        # Ahora que tenemos el ΔE, podemos decidir si aceptamos o no el movimiento
        # La condición básicamente es que para ΔE <= 0, aceptamos el movimiento, ya que de ser así la probabilidad de aceptación es 1.
        # Si ΔE > 0, aceptamos el movimiento con probabilidad p = exp(-ΔE/T), y lo más eficiente es generar un número aleatorio entre 0 y 1 y comparar con p,
        # ya que si el número aleatorio es menor o igual que p, aceptamos el movimiento.
        if delta_E <= 0 or np.random.random() < np.exp(-delta_E / T):
            # Intercambiar espines
            config[i, j], config[ni, nj] = config[ni, nj], config[i, j]
    


In [11]:
# ─── Celda 5: Función del bucle Monte Carlo y recolección de datos con HDF5 ────────────

def run_monte_carlo(L, J, T, n_sweeps, rng, config, thin, destino):

    # Inicializar arrays para almacenar energía y magnetización
    energies = np.zeros(n_sweeps + 1)
    magnetizations = np.zeros(n_sweeps + 1)

    # Parámetros de guardado
    n_saved = (n_sweeps // thin) + 1

    with h5py.File(destino, 'w') as f:
        # Dataset para las configuraciones: snapshots × L × L, dtype int8
        dataset = f.create_dataset(
            'configs',                      # 1. Nombre del dataset dentro del archivo HDF5
            shape=(n_saved, L, L),          # 2. Dimensiones: n_saved muestras de matrices L×L     
            dtype='i1',                     # 3. Tipo de dato: int8 (espines ±1)
            compression='gzip',             # 4. Compresión: algoritmo gzip
            compression_opts=4,             # 5. Nivel de compresión (1=rápido/menos compacto … 9=lento/máximo)
            chunks=(1, L, L),               # 6. Fragmentación (“chunking”): cada bloque es una matriz L×L
        )
        # Metadatos
        f.attrs['J'] = J
        f.attrs['T'] = T
        f.attrs['L'] = L
        f.attrs['n_sweeps'] = n_sweeps
        f.attrs['thin'] = thin

        # Medir estado inicial
        energies[0] = energy(config, J)
        magnetizations[0] = magnetization(config)
        # Guardar configuración inicial ds[0]
        dataset[0, :, :] = config

        # Barrido Monte Carlo
        start_time = time.time()
        idx = 1  # índice para guardar snapshots
        for sweep in tqdm(range(1, n_sweeps + 1), desc='MC Sweeps'):  # Esto es una simple barra de progreso, nada más
            # Ahora podemos barrer la red para elegir el par de espines a intercambiar.
            sweep_kawasaki(config, L, J, T)
            # Registrar observables
            energies[sweep] = energy(config, J)
            magnetizations[sweep] = magnetization(config)

            # Almacenar las configuraciones si toca
            if sweep % thin == 0:
                dataset[idx, :, :] = config
                idx += 1

        end_time = time.time()

        print(f"Simulación completada en {end_time - start_time:.2f} s")

# Plotear los datos importantes

def plot_observables(energies, magnetizations, n_sweeps, destino):
    # ─── Acceptance rate ────────────────────────────────

    # 1) Cargar todos los frames desde el HDF5
    with h5py.File(destino, 'r') as f:
        frames = f['configs'][:]    # np.ndarray (nframes, H, W)

    # 2) Calcular la aceptación
    acceptance = calculate_acceptance(frames)

    # 3) Representar la evolución de la tasa de aceptación
    sweeps = np.arange(1, len(acceptance) + 1)
    plt.figure(figsize=(6, 4))
    plt.plot(sweeps, acceptance, linestyle='-')
    plt.xlabel('Sweep')
    plt.ylabel('Acceptance rate')
    plt.title('Evolución de la tasa de aceptación (Kawasaki)')
    plt.grid(True)
    plt.tight_layout()

    # 4) Guardar la figura
    plt.savefig(os.path.join(destino, 'acceptance_rate.png'), dpi=300, bbox_inches='tight')

    # ─── Energía ────────────────────────────────

    n_sweeps_array = np.arange(n_sweeps + 1)

    plt.figure(figsize=(6, 4))
    plt.plot(n_sweeps_array, energies, linestyle='-')
    plt.xlabel('Sweep')
    plt.ylabel('Energía')
    plt.title('Energía del sistema (Kawasaki)')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(destino, 'energy.png'), dpi=300, bbox_inches='tight')
    
    # ─── Magnetización ────────────────────────────────

    plt.figure(figsize=(6, 4))
    plt.plot(n_sweeps_array, magnetizations, linestyle='-')
    plt.xlabel('Sweep')
    plt.ylabel('Magnetización')
    plt.title('Magnetización del sistema (Kawasaki)')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(destino, 'magnetization.png'), dpi=300, bbox_inches='tight')

In [ ]:
# Celda: pipeline GPU-BOUND con NVENC a partir de HDF5

def generate_video_from_hdf5(HDF5_FILE, DATASET, FILE_OUT, GPU_ID, INTERVAL, TARGET_W, TARGET_H, MIN_SIDE, destino):

    # 1) Cargar datos --------------------------------------------------------------
    with h5py.File(os.path.join(destino, HDF5_FILE), 'r') as f:
        frames = f[DATASET][:]
    nframes, h0, w0 = frames.shape
    fps = 1000.0 / INTERVAL
    print(f"→ {nframes} frames ({w0}×{h0}px) @ {fps:.1f} fps")

    # 2) Calcular resolución de salida --------------------------------------------
    w_out, h_out = w0, h0
    if TARGET_W and not TARGET_H:
        scale = TARGET_W / w0
        w_out = TARGET_W
        h_out = int(round(h0 * scale))
    elif TARGET_H and not TARGET_W:
        scale = TARGET_H / h0
        h_out = TARGET_H
        w_out = int(round(w0 * scale))
    elif TARGET_W and TARGET_H:
        w_out, h_out = TARGET_W, TARGET_H

    # Asegurar mínimo NVENC
    if min(w_out, h_out) < MIN_SIDE:
        factor = math.ceil(MIN_SIDE / min(w_out, h_out))
        w_out *= factor
        h_out *= factor
        print(f"⚠️ Redimensionado extra para mínimo NVENC ({MIN_SIDE}px)")

    # Redondear a par
    w_out = (w_out // 2) * 2
    h_out = (h_out // 2) * 2
    if (w_out, h_out) != (w0, h0):
        print(f"🔧 Escalando: {w0}×{h0} → {w_out}×{h_out}")
    vf_filter = ["-vf", f"scale={w_out}:{h_out}:flags=neighbor"] if (w_out, h_out) != (w0, h0) else []

    # 3) Detectar NVENC ------------------------------------------------------------
    encoders = subprocess.run(
        ["ffmpeg", "-hide_banner", "-encoders"],
        capture_output=True, text=True
    ).stdout
    if "h264_nvenc" in encoders:
        print("✅ h264_nvenc detectado → GPU")
        video_opts = [
            "-c:v", "h264_nvenc", "-gpu", str(GPU_ID),
            "-preset", "p1", "-profile:v", "high444p", "-pix_fmt", "yuv444p"
        ]
    else:
        print("⚠️ NVENC no disponible → libx264 (CPU)")
        video_opts = ["-c:v", "libx264", "-preset", "veryslow", "-crf", "0", "-pix_fmt", "yuv420p"]

    # 4) Comando FFmpeg ------------------------------------------------------------
    cmd = (
        ["ffmpeg", "-y",
        "-f", "rawvideo", "-pix_fmt", "rgb24",
        "-s", f"{w0}x{h0}", "-r", str(fps), "-i", "-",
        "-progress", "pipe:1", "-loglevel", "error"]
        + vf_filter + video_opts + [f"{os.path.join(destino, FILE_OUT)}.mp4"]
    )
    print("FFmpeg:", " ".join(cmd))

    # 5) Lanzar FFmpeg -------------------------------------------------------------
    proc = subprocess.Popen(
        cmd,
        stdin = subprocess.PIPE,
        stdout = subprocess.PIPE,
        stderr = subprocess.PIPE,
        bufsize = 0
    )

    # 6) Barra de codificación -----------------------------------------------------
    pbar_enc = tqdm(total=nframes, desc="🛠️ Codificando", unit="frame")
    def _watch():
        re_time = re.compile(rb"out_time_ms=(\d+)")
        re_fr   = re.compile(rb"frame=(\d+)")
        while True:
            line = proc.stdout.readline()
            if not line:
                break
            m = re_fr.search(line) or re_time.search(line)
            if m:
                val = int(m.group(1))
                done = val if b"frame" in line else min(nframes, int(round(val * fps / 1000)))
                pbar_enc.n = done
                pbar_enc.refresh()

    threading.Thread(target=_watch, daemon=True).start()

    # 7) Enviar frames -------------------------------------------------------------
    with tqdm(total=nframes, desc="📤 Enviando frames", unit="frame") as pbar_in:
        for frame in frames:
            # Convertir de Ising (-1,+1) a [0,255] y a RGB
            rgb = np.repeat(((frame + 1) * 127.5).astype(np.uint8)[..., None], 3, axis=2)
            proc.stdin.write(rgb.tobytes())
            pbar_in.update(1)

    proc.stdin.close()
    proc.wait()
    pbar_enc.n = nframes
    pbar_enc.refresh()
    pbar_enc.close()

    print(f"🎉 Vídeo generado: {os.path.join(destino, FILE_OUT)}.mp4 ({w_out}×{h_out})")


In [ ]:
# ── Celda 7: Ejecución múltiple de las diferentes simulaciones ────────────────────────────────────────────────────









# ── Parámetros de usuario para la generación del vídeo ─────────────────────────────────────────────────────

HDF5_FILE = "configs.h5"        # Nombre del archivo HDF5 con las configuraciones
DATASET   = "configs"           # Nombre del dataset dentro del archivo HDF5
FILE_OUT  = "simulacion"        # Nombre del archivo de salida (sin extensión ni ruta)
GPU_ID    = 0                   # 0 = tu NVIDIA 4050
INTERVAL  = 50                  # ms entre frames → fps = 1000/INTERVAL
TARGET_W  = 1440                # ancho deseado; None para mantener original
TARGET_H  = None                # alto deseado; None para mantener original
MIN_SIDE  = 160                 # mínimo seguro para NVENC (≥ 145 y par)

# ──────────────────────────────────────────────────────────────────────────────

generate_video_from_hdf5(HDF5_FILE, DATASET, FILE_OUT, GPU_ID, INTERVAL, TARGET_W, TARGET_H, MIN_SIDE)
